In [42]:
import pandas as pd
import numpy as np
import xgboost as xgb
from xgboost import plot_importance
from xgboost import XGBClassifier
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score, f1_score, confusion_matrix

import warnings
warnings.filterwarnings('ignore')

데이터 불러오기 

data1 : phenotype_1000

data2 : KCPS

data3 : phenotype_1000 + KCPS

In [43]:
data1=pd.read_csv('C:/Users/dudtj/OneDrive - 숭실대학교 - Soongsil University/Desktop/CL/python/동계인턴십_Data_1000/phenotype_1000.txt',engine="python",sep=" ")

In [44]:
data2=pd.read_csv('C:/Users/dudtj/OneDrive - 숭실대학교 - Soongsil University/Desktop/CL/python/동계인턴십_Data_1000/KCPS_r2_0.01_1000.csv')

In [45]:
for i in range(2,15190):
    data2.columns.values[i]=i

In [46]:
data1=data1.loc[:,['FID','LUNG']]

In [47]:
import matplotlib.pyplot as plt 
import seaborn as sns

In [48]:
data1 = pd.concat([data1,data2],axis=1)

In [49]:
data1.dropna(inplace=True)
data1.drop('FID',axis=1,inplace=True)
data1.drop('IID',axis=1,inplace=True)

In [50]:
#corr = data1.corr(method='pearson')
#colormap = plt.cm.PuBu 
#plt.figure(figsize=(15,15)) 
#plt.title("Person Correlation of Features", y = 1.05, size = 15) 
#sns.heatmap(data1.astype(float).corr(),fmt='.2f', linewidths = 0.1, vmax = 1.0, square = True, cmap = colormap, linecolor = "white", annot = True, annot_kws = {"size" : 10})

In [51]:
data1

,LUNG,2,3,4,5,6,7,8,9,10,...,15180,15181,15182,15183,15184,15185,15186,15187,15188,15189
0,0,0,1,0,2,1,1,1,0,2,...,0,2,0,2,0,1,1,1,0,2
1,0,0,2,0,1,0,1,2,0,2,...,0,1,0,1,0,0,1,0,2,0
2,0,0,0,0,1,2,0,1,0,1,...,0,2,0,0,0,2,0,2,0,0
3,0,0,1,0,0,0,1,1,0,2,...,0,0,0,0,0,0,1,1,1,0
4,0,1,0,0,1,1,2,1,1,1,...,0,2,0,2,0,1,0,1,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,0,0,1,0,2,1,1,1,1,2,...,0,0,0,1,0,1,2,0,0,1
996,0,2,2,0,2,0,1,0,1,1,...,0,2,0,1,0,0,0,2,0,2
997,0,1,1,0,0,0,2,1,0,0,...,0,1,0,2,0,2,1,1,1,1
998,0,0,0,0,2,0,1,1,1,0,...,0,2,0,0,0,1,1,1,0,0


In [52]:
y_df = data1.iloc[:, 0]

In [53]:
data1.drop('LUNG',axis=1,inplace=True)
X_df = data1

In [54]:
X_df

,2,3,4,5,6,7,8,9,10,11,...,15180,15181,15182,15183,15184,15185,15186,15187,15188,15189
0,0,1,0,2,1,1,1,0,2,0,...,0,2,0,2,0,1,1,1,0,2
1,0,2,0,1,0,1,2,0,2,2,...,0,1,0,1,0,0,1,0,2,0
2,0,0,0,1,2,0,1,0,1,1,...,0,2,0,0,0,2,0,2,0,0
3,0,1,0,0,0,1,1,0,2,1,...,0,0,0,0,0,0,1,1,1,0
4,1,0,0,1,1,2,1,1,1,2,...,0,2,0,2,0,1,0,1,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,0,1,0,2,1,1,1,1,2,1,...,0,0,0,1,0,1,2,0,0,1
996,2,2,0,2,0,1,0,1,1,1,...,0,2,0,1,0,0,0,2,0,2
997,1,1,0,0,0,2,1,0,0,1,...,0,1,0,2,0,2,1,1,1,1
998,0,0,0,2,0,1,1,1,0,1,...,0,2,0,0,0,1,1,1,0,0


In [55]:
#y_df =data1['LUNG']
#X_df =data1.drop('LUNG', axis=1)

In [56]:
import numpy as np
import  pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score # 평가를 위함
from sklearn.metrics import f1_score, confusion_matrix, precision_recall_curve, roc_curve
from sklearn.preprocessing import StandardScaler, Binarizer

from sklearn.tree import DecisionTreeClassifier 
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import AdaBoostClassifier
from xgboost import XGBClassifier

In [57]:
from sklearn.linear_model import Lasso

In [58]:
#X_df_columns=X_df.iloc[:,lasso_low.coef_!=0].columns

In [59]:
#X_df_columns

In [60]:
#X_df=data1.loc[:,X_df_columns]

In [61]:
X_df

,2,3,4,5,6,7,8,9,10,11,...,15180,15181,15182,15183,15184,15185,15186,15187,15188,15189
0,0,1,0,2,1,1,1,0,2,0,...,0,2,0,2,0,1,1,1,0,2
1,0,2,0,1,0,1,2,0,2,2,...,0,1,0,1,0,0,1,0,2,0
2,0,0,0,1,2,0,1,0,1,1,...,0,2,0,0,0,2,0,2,0,0
3,0,1,0,0,0,1,1,0,2,1,...,0,0,0,0,0,0,1,1,1,0
4,1,0,0,1,1,2,1,1,1,2,...,0,2,0,2,0,1,0,1,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,0,1,0,2,1,1,1,1,2,1,...,0,0,0,1,0,1,2,0,0,1
996,2,2,0,2,0,1,0,1,1,1,...,0,2,0,1,0,0,0,2,0,2
997,1,1,0,0,0,2,1,0,0,1,...,0,1,0,2,0,2,1,1,1,1
998,0,0,0,2,0,1,1,1,0,1,...,0,2,0,0,0,1,1,1,0,0


In [62]:
# random_state = 156 : 다음에도 같은 데이터를 train, test로 분리 하는 옵션
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X_df,y_df,test_size=0.2,random_state=156,stratify=y_df)

In [63]:
lasso = Lasso(alpha=0.001, max_iter=100000).fit(X_train, y_train)

print("훈련 세트의 정확도 : {:.2f}".format(lasso.score(X_train, y_train)))
print("테스트 세트의 정확도 : {:.2f}".format(lasso.score(X_test, y_test)))
print("사용한 특성의 수 : {}".format(np.sum(lasso.coef_ != 0)))
print("사용한 max_iter : {}".format(lasso.n_iter_))

훈련 세트의 정확도 : 0.99
테스트 세트의 정확도 : -0.22
사용한 특성의 수 : 726
사용한 max_iter : 258


In [87]:
lasso = Lasso(alpha=0.0001, max_iter=100000).fit(X_train, y_train)

print("훈련 세트의 정확도 : {:.2f}".format(lasso.score(X_train, y_train)))
print("테스트 세트의 정확도 : {:.2f}".format(lasso.score(X_test, y_test)))
print("사용한 특성의 수 : {}".format(np.sum(lasso.coef_ != 0)))
print("사용한 max_iter : {}".format(lasso.n_iter_))

훈련 세트의 정확도 : 0.71
테스트 세트의 정확도 : -0.43
사용한 특성의 수 : 133
사용한 max_iter : 10


In [65]:
X_train_columns=X_train.iloc[:,lasso.coef_!=0].columns

In [68]:
X_train_columns

Index([   35,   151,   304,   314,   408,   506,   579,   678,  1054,  1065,
       ...
       13896, 13932, 14039, 14064, 14282, 14295, 14494, 14809, 14939, 15171],
      dtype='object', length=133)

In [70]:
X_test = X_test[X_train_columns]
X_train = X_train[X_train_columns]

In [73]:
def get_clf_eval(y_test, pred=None, pred_proba=None):
    confusion = confusion_matrix(y_test, pred) # 오차 행렬 (평가 지표)
    accuracy = accuracy_score(y_test, pred) # 정확도
    precision = precision_score(y_test, pred,average='micro') # 정밀도
    recall = recall_score(y_test, pred,average='micro') # 재현율
    f1 = f1_score(y_test, pred,average='micro') # F1 (정밀도, 재현율 적정 수준 : 1에 가까울 수록 좋음)
    roc_auc = roc_auc_score(y_test, pred_proba,multi_class='ovo') # AUC (1에 가까울 수록 좋음)
    
    print('오차 행렬\n {}'.format(confusion))
    print('정확도 : {0:.4f}, 정밀도 : {1:.4f}, 재현율 : {2:.4f}, F1 : {3:.4f}, AUC: {4:.4f}'
         .format(accuracy, precision, recall, f1, roc_auc))

In [74]:
def precision_recall_curve_plot(y_test=None, pred_proba_c1=None):
    # threshold ndarray와 이 threshold에 따른 정밀도, 재현율 ndarray 추출. 
    precisions, recalls, thresholds = precision_recall_curve( y_test, pred_proba_c1)
    
    # X축을 threshold값으로, Y축은 정밀도, 재현율 값으로 각각 Plot 수행. 정밀도는 점선으로 표시
    plt.figure(figsize=(8,6))
    threshold_boundary = thresholds.shape[0]
    plt.plot(thresholds, precisions[0:threshold_boundary], linestyle='--', label='precision')
    plt.plot(thresholds, recalls[0:threshold_boundary],label='recall')
    
    # threshold 값 X 축의 Scale을 0.1 단위로 변경
    start, end = plt.xlim()
    plt.xticks(np.round(np.arange(start, end, 0.1),2))
    
    # x축, y축 label과 legend, 그리고 grid 설정
    plt.xlabel('Threshold value'); plt.ylabel('Precision and Recall value')
    plt.legend(); plt.grid()
    plt.show()

In [75]:
#logistic Regression
logistic = LogisticRegression()
logistic.fit(X_train, y_train)
pred = logistic.predict(X_test)
pred_proba = logistic.predict_proba(X_test)[:,1]
get_clf_eval(y_test, pred, pred_proba)

오차 행렬
 [[193   0]
 [  7   0]]
정확도 : 0.9650, 정밀도 : 0.9650, 재현율 : 0.9650, F1 : 0.9650, AUC: 0.4189


In [76]:
#random forest
random = RandomForestClassifier()
random.fit(X_train, y_train)
pred = random.predict(X_test)
pred_proba = random.predict_proba(X_test)[:,1]
get_clf_eval(y_test, pred, pred_proba)

오차 행렬
 [[193   0]
 [  7   0]]
정확도 : 0.9650, 정밀도 : 0.9650, 재현율 : 0.9650, F1 : 0.9650, AUC: 0.4833


In [77]:
#tree
tree = RandomForestClassifier()
tree.fit(X_train, y_train)
pred = tree.predict(X_test)
pred_proba = tree.predict_proba(X_test)[:,1]
get_clf_eval(y_test, pred, pred_proba)

오차 행렬
 [[193   0]
 [  7   0]]
정확도 : 0.9650, 정밀도 : 0.9650, 재현율 : 0.9650, F1 : 0.9650, AUC: 0.3597


In [78]:
#adaboost
adaboost = AdaBoostClassifier()
adaboost.fit(X_train, y_train)
pred = adaboost.predict(X_test)
pred_proba = adaboost.predict_proba(X_test)[:,1]
get_clf_eval(y_test, pred, pred_proba)

오차 행렬
 [[193   0]
 [  7   0]]
정확도 : 0.9650, 정밀도 : 0.9650, 재현율 : 0.9650, F1 : 0.9650, AUC: 0.4893


In [79]:
#xgboost
xgboost = XGBClassifier()
xgboost.fit(X_train, y_train)
pred = xgboost.predict(X_test)
pred_proba = xgboost.predict_proba(X_test)[:,1]
get_clf_eval(y_test, pred, pred_proba)

[19:19:41] WARNING: ..\src\learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
오차 행렬
 [[193   0]
 [  7   0]]
정확도 : 0.9650, 정밀도 : 0.9650, 재현율 : 0.9650, F1 : 0.9650, AUC: 0.4478


In [82]:
from lightgbm import LGBMClassifier
lgbm = LGBMClassifier(n_estimators=400)
lgbm.fit(X_train, y_train)
pred = lgbm.predict(X_test)
pred_proba = lgbm.predict_proba(X_test)[:,1]
get_clf_eval(y_test, pred, pred_proba)


오차 행렬
 [[193   0]
 [  7   0]]
정확도 : 0.9650, 정밀도 : 0.9650, 재현율 : 0.9650, F1 : 0.9650, AUC: 0.4182
